In [ ]:
%%time

import pickle
import sqlite3
import numpy as np
import pandas as pd
from sqlite3 import Error

#############################
#
# load the data
#
#############################



with open('cleaned_sentences.pickle', 'rb') as handle:
    cleaned_sentences = pickle.load(handle) 
    
    
# look at http://pythondata.com/category/text-analytics/